In [67]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


In [55]:
df = pd.read_csv("data/europisco.csv")

In [56]:
df.drop(columns=["Email Address", "Cuál es tu correo? (opcional)"], inplace=True)

In [57]:
df.rename(
    columns={
        "Timestamp": "timestamp",
        "Cúal es la dirección exacta (o más aproximada) dónde viste el pisco?": "address_raw",
        "En qué país?": "country",
        "Qué era específicamente?": "pisco_type",
        "Cúanto costaba aproximadamente (en euros)?": "price",
        "Cuál es tu nombre? (opcional)": "name",
        "Probable address": "address_clean"
    },
    inplace=True
)

In [58]:
df.dropna(subset=['address_clean'], inplace=True)

In [59]:
df.reset_index(inplace=True, drop=True)

In [60]:
df

,timestamp,address_raw,country,pisco_type,price,name,address_clean
0,6/8/2024 22:34:22,"Rda. de Sant Antoni, 15, L'Eixample, 08011 Bar...",España,Botella,17.0,Joaquín Arretx,"Rda. de Sant Antoni, 15, L'Eixample, 08011 Bar..."
1,6/14/2024 17:40:03,"Ham 31, 9000 Gent",Bélgica,Botella,25.0,Sebastian,"Ham 31, 9000 Gent"
2,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Prenzlauer Promenade 192, 13189 Berlin, Germany"
3,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Kantstraße 42, 10625 Berlin, Germany"
4,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Berliner Str. 78, 14169 Berlin, Germany"
5,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Kolonnenstraße 30B, 10829 Berlin, Germany"
6,6/14/2024 17:58:38,Bar Juana Brava en Salamanca,España,Piscola,6.0,Fabián,"Calle de Van Dyck, 13, 37005 Salamanca"
7,6/14/2024 18:36:46,"Carrer de Matias Perelló, 52, L'Eixample, 4600...",España,Botella,15.0,Zeroday,"Carrer de Matias Perelló, 52, L'Eixample, 4600..."
8,6/14/2024 19:15:04,Algunos Systembolaget Suecia,Suecia,Botella,25.0,Mauricio,"Drottninggatan 45, 111 21 Stockholm, Sweden"
9,6/14/2024 20:34:17,"Industriepark-Noord 35, 9100 Sint-Niklaas",Bélgica,Botella,50.0,Milena,"Industriepark-Noord 35, 9100 Sint-Niklaas"


In [61]:
geolocator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Create new columns for latitude and longitude
df['location'] = df['address_clean'].apply(geocode)
df['latitude'] = df['location'].apply(lambda loc: loc.latitude if loc else None)
df['longitude'] = df['location'].apply(lambda loc: loc.longitude if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*("Rda. de Sant Antoni, 15, L'Eixample, 08011 Barcelona",), **{}).
Traceback (most recent call last):
  File "/home/jarretx/europisco/venv/lib/python3.10/site-packages/geopy/geocoders/base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
  File "/home/jarretx/europisco/venv/lib/python3.10/site-packages/geopy/adapters.py", line 472, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
  File "/home/jarretx/europisco/venv/lib/python3.10/site-packages/geopy/adapters.py", line 500, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 403

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/jarretx/europisco/venv/lib/python3.10/site-packages/geopy/extra/rate_limiter.py", line 136, in _retries_gen
    yield i  # Run the function.
  F

In [63]:
df.query("location.isnull()")

,timestamp,address_raw,country,pisco_type,price,name,address_clean,location,latitude,longitude
0,6/8/2024 22:34:22,"Rda. de Sant Antoni, 15, L'Eixample, 08011 Bar...",España,Botella,17.0,Joaquín Arretx,"Rda. de Sant Antoni, 15, L'Eixample, 08011 Bar...",None,NaN,NaN
6,6/14/2024 17:58:38,Bar Juana Brava en Salamanca,España,Piscola,6.0,Fabián,"Calle de Van Dyck, 13, 37005 Salamanca",None,NaN,NaN
18,6/15/2024 14:18:11,"Wine House, Rda. de Sant Antoni, 15, L'Eixampl...",España,Botella,23.0,NaN,"Wine House, Rda. de Sant Antoni, 15, L'Eixampl...",None,NaN,NaN
21,6/15/2024 15:49:08,"Bar L'Amistat, RAMON Y CAJAL 58, TORRIJOS 13, ...",España,Piscola,8.0,Joaquín Arretx,"RAMON Y CAJAL 58, TORRIJOS 13, 08012 Barcelona",None,NaN,NaN
23,6/16/2024 18:26:28,"Gran Vía de Don Diego López de Haro, 9, Abando...",España,Botella,13.0,Nico,"Gran Vía de Don Diego López de Haro, 9, Abando...",None,NaN,NaN
30,6/16/2024 19:44:25,El corte inglés Valencia Centro,España,Botella,35.0,"Eduardo ""Piscola"" Segovia","Calle del Pintor Sorolla, 26, 46002, Ciutat Ve...",None,NaN,NaN
32,6/16/2024 20:23:57,Barrio de gracia Barcelona,España,"Piscola, Pisco Sour",15.0,Bar kiltro,"Carrer del Montseny, 13, Local 2, Gracia, 0801...",None,NaN,NaN


In [65]:
df.loc[0, "latitude"] = 41.38006562275075
df.loc[0, "longitude"] = 2.163010839291413
df.loc[6, "latitude"] = 40.972546300325746
df.loc[6, "longitude"] = -5.658419387328101
df.loc[18, "latitude"] = 41.38006562275075
df.loc[18, "longitude"] = 2.163010839291413
df.loc[21, "latitude"] = 41.40293279969603
df.loc[21, "longitude"] = 2.159256096961093
df.loc[23, "latitude"] = 43.26163372566617
df.loc[23, "longitude"] = -2.929120933831521
df.loc[30, "latitude"] = 39.47078392527955
df.loc[30, "longitude"] = -0.37138733009770314
df.loc[32, "latitude"] = 41.40191119891952
df.loc[32, "longitude"] = 2.154507805637604

In [69]:
df.head(5)

,timestamp,address_raw,country,pisco_type,price,name,address_clean,location,latitude,longitude
0,6/8/2024 22:34:22,"Rda. de Sant Antoni, 15, L'Eixample, 08011 Bar...",España,Botella,17.0,Joaquín Arretx,"Rda. de Sant Antoni, 15, L'Eixample, 08011 Bar...",None,41.380066,2.163011
1,6/14/2024 17:40:03,"Ham 31, 9000 Gent",Bélgica,Botella,25.0,Sebastian,"Ham 31, 9000 Gent","(31, Huidevetterskaai, Waterwijk, Gent, Oost-V...",51.060588,3.728483
2,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Prenzlauer Promenade 192, 13189 Berlin, Germany","(Denns BioMarkt, 192, Prenzlauer Promenade, Pr...",52.551672,13.429937
3,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Kantstraße 42, 10625 Berlin, Germany","(Mitte Meer, 42, Kantstraße, Charlottenburg, C...",52.506538,13.312701
4,6/14/2024 17:48:40,Mitte Meer Shop (Berlin),Alemania,Botella,18.0,Bernardo Suárez,"Berliner Str. 78, 14169 Berlin, Germany","(Mitte Meer, 78, Berliner Straße, Zehlendorf, ...",52.439565,13.276804


In [73]:
# Initialize the map centered around Europe
m = folium.Map(location=[54.5260, 15.2551], zoom_start=4)

# Function to add markers to the map
def add_marker(row):
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Tipo: {row['pisco_type']}<br>Precio: {row['price']}<br>Subido por: {row['name']}<br>Dirección: {row['address_clean']}",
    ).add_to(m)

# Apply the function to each row in the dataframe
df.apply(add_marker, axis=1)

# Save the map to an HTML file
m.save('out/map.html')